# Neuroscience using `fastplotlib` :D

This notebook will build up a complex visualization using `fastplotlib`, in conjunction with other Python neuroscience packages, to exemplify how `fastplotlib` can be a powerful tool in analysis and visualization of neural data!

Packages involved: 
- `mesmerize-core`
- `pynapple`
- `animal-soup`
- `fastplotlib`
- `CaImAn`

In [1]:
import fastplotlib as fpl
from mesmerize_core import *
import pynapple as nap
import numpy as np

2023-10-31 16:33:02.518504: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 16:33:02.518532: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 16:33:02.518552: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 16:33:02.522947: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 16:33:03.054969: W tensorflow/compiler/

### Opening behavior videos using **Lazy Loading** and visualizing them using `fastplotlib`.

#### **Lazy Loading**: strategy to conserve RAM by dynamically loading in frames for visualization only as needed 

Behavioral and neural data collected during experiments can be up to **terabytes** in size, making it impossible to load all of the data into RAM at once for visualization and analysis. ***Lazy Loading*** allows us to bypass our memory space constraint without overburdening our resources!

### Visualizing some results of running `CaImAn` via `mesmerize-core` on calcium imaging data.

### Creating an interactive visualization to perform calcium imaging analysis.

### Syncing behavior and calcium data using `pynapple`.